In [28]:
distances = [
                [0, 20, 14],
                [20, 0, 6],
                [14, 6, 0],
            ]

class Location:
    def __init__(self, indx, name):
        self.indx = indx
        self.name = name
        self.samples = []
        
    def distance_to(self, location):
        return distances[self.indx][location.indx]
    
    def add_sample(self, sample):
        self.samples.append(sample)
        
    @property
    def time_to_deadline(self):
        return min([sample.time_to_deadline for sample in self.samples] + [float('inf')])
    
    def __str__(self):
        return self.name
        
lab = Location(0, 'lab')

class Sample:
    def __init__(self, name, location, time_to_deadline, destination=None):
        self.name = name
        self.location = location
        self.time_to_deadline = time_to_deadline
        self.destination = destination or lab
        self.location.add_sample(self)
        
    def pass_time(self, minutes):
        self.time_to_deadline -= minutes
        if self.is_late() and not self.has_arrived():
            return False
        return True
    
    def is_late(self):
        return self.time_to_deadline < 0
    
    def has_arrived(self):
        return self.location == self.destination
    
    def in_transit(self):
        return not isinstance(self.location, Location)
    
    def __str__(self):
        if self.has_arrived():
            return f"Sample {self.name} arrived."
        if self.is_late():
            f"Sample {self.name} is late! at {self.location.name}"
        return f"Sample {self.name} at {self.location.name} with {self.time_to_deadline} minutes to deadline"


class Porter:
    def __init__(self, name):
        self.name = name
        self.distance_to_location = 0
        self.location = lab
        self.samples = []

    def move_to(self, new_location):
        assert self.distance_to_location == 0
        self.distance_to_location = self.location.distance_to(new_location)
        self.location = new_location
        
    def pick_up_sample(self, sample):
        assert self.location == sample.location and self.distance_to_location == 0
        self.samples.append(sample)
        sample.location = self
        self.location.samples.remove(sample)
        
    def pick_up_samples(self):
        if self.distance_to_location == 0:
            for sample in self.location.samples:
                if not sample.has_arrived():
                    self.pick_up_sample(sample)
        
    def drop_samples(self):
        if self.distance_to_location == 0:
            for sample in self.samples:
                if sample.destination == self.location:
                    sample.location = self.location
                    self.samples.remove(sample)
                    
    def time_to_deadline(self):
        return min([sample.time_to_deadline for sample in self.samples] + [float('inf')])
    
    def in_transit(self):
        return self.distance_to_location > 0
        
    def pass_time(self, minutes):
        self.distance_to_location = max(self.distance_to_location - minutes, 0)
        
    def __str__(self):
        return f"Porter {self.distance_to_location} minutes to {self.location} with {[sample.name for sample in self.samples]} samples"

In [36]:
class Hospital:
    def __init__(self):
        self.time = 0
        self.porters = [Porter("Joe")]
        self.locations = [lab, Location(1, 'ward_1'), Location(2, 'ward_2')]
        self.samples = [Sample("KGD12", self.locations[1], 45), Sample("KG24", self.locations[2], 120)]
        
    def increment_time(self, minutes=1):
        self.time += minutes
        for porter in self.porters:
            porter.pass_time(minutes)
        for sample in self.samples:
            sample.pass_time(minutes)
            
    def locations_by_urgency(self):
        return sorted(self.locations, key=lambda sample: sample.time_to_deadline)
            
    def __str__(self):
        rtn = f"{self.time} minutes passed\n"

        for porter in self.porters:
            rtn += str(porter) + "\n"
        for sample in self.samples:
            rtn += str(sample) + "\n"
        return rtn

In [37]:
hospital = Hospital()

while hospital.time < 240:
             
    #Porters drop and pick-up samples
    for porter in hospital.porters:
        porter.pick_up_samples()
        porter.drop_samples()
    
    
    urgency_locations = hospital.locations_by_urgency()
    
    for porter in hospital.porters:
        if not porter.in_transit():
            if porter.samples:
                porter.move_to(porter.samples[-1].destination)
            else:
                location = urgency_locations.pop(0)
                porter.move_to(location)

    #Increment times
    hospital.increment_time()
    
    if hospital.time%3 == 0:
        print(hospital)
        
    

    
    
    

3 minutes passed
Porter 17 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 42 minutes to deadline
Sample KG24 at ward_2 with 117 minutes to deadline

6 minutes passed
Porter 14 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 39 minutes to deadline
Sample KG24 at ward_2 with 114 minutes to deadline

9 minutes passed
Porter 11 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 36 minutes to deadline
Sample KG24 at ward_2 with 111 minutes to deadline

12 minutes passed
Porter 8 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 33 minutes to deadline
Sample KG24 at ward_2 with 108 minutes to deadline

15 minutes passed
Porter 5 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 30 minutes to deadline
Sample KG24 at ward_2 with 105 minutes to deadline

18 minutes passed
Porter 2 minutes to ward_1 with [] samples
Sample KGD12 at ward_1 with 27 minutes to deadline
Sample KG24 at ward_2 with 102 minutes to deadline

21 minutes passe

In [13]:
a = [1,2,3,6,4,5]

In [7]:
sorted(a)

[1, 2, 3]

In [11]:
a.remove(1)

In [14]:
for num in a:
    if num > 4:
        a.remove(num)

In [15]:
a

[1, 2, 3, 4]